In [1]:
!pip install -qU langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 29.5 MB/s eta 0:00:00


In [15]:
from google.colab import userdata
my_key=userdata.get("my_key2")

###1. Data Ingestion
###2. Data Reterival
###3. Data Generation

In [3]:
!pip install -qU langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00


In [4]:
from langchain.document_loaders import TextLoader

In [5]:
loader=TextLoader('/content/state_of_the_union.txt',encoding="utf8")
doc=loader.load()

In [6]:
doc[0].page_content[:50]

'Madam Speaker, Madam Vice President, our First Lad'

### chunking of data


In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)

In [8]:
text_chunks=text_splitter.split_documents(doc)
print("first chunk : \n",text_chunks[0].page_content)
print("-"*50)
print("second chunk : \n",text_chunks[1].page_content)

first chunk : 
 Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny.
--------------------------------------------------
second chunk : 
 Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination

In [9]:
!pip install -qU  langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [10]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 13.4 MB/s eta 0:00:00


In [16]:
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [36]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=my_key)
# embeddings.embed_query("What's our Q1 revenue?")

In [21]:
vectorstore=FAISS.from_documents(text_chunks, embeddings)

In [22]:
retriever=vectorstore.as_retriever()

In [24]:
from langchain.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [25]:
prompt=ChatPromptTemplate.from_template(template)

In [29]:
!pip install -qU "langchain[google-genai]"

In [33]:
from langchain.chat_models import init_chat_model
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

llm=init_chat_model(model="gemini-2.5-flash",google_api_key=my_key,model_provider="google_genai")
output_parser=StrOutputParser()

In [54]:
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

In [55]:
rag_chain.invoke("How is the United States supporting Ukraine economically and militarily?")

"The United States is supporting Ukraine through both economic and military assistance. Militarily, the U.S. is providing direct assistance to aid Ukraine in its fight for freedom. Economically, the United States has given more than $1 billion in direct assistance to Ukraine. Additionally, the U.S., along with its allies, is enforcing powerful economic sanctions targeted at Russia's economy. The United States has also collaborated with 30 other countries to release 60 million barrels of oil from reserves around the world. These measures are intended to help the Ukrainian people defend their country and alleviate their suffering."

In [56]:
rag_chain.invoke("What action is the U.S. taking to address rising gas prices?")

'To address rising gas prices, the U.S. is taking action by releasing oil from its Strategic Petroleum Reserve. Specifically, America is leading an effort to release 30 million barrels from its own reserve. Additionally, the United States has collaborated with 30 other countries to collectively release 60 million barrels of oil from reserves around the world. These steps are intended to help alleviate gas prices for consumers at home.'